<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/AOCC_gemini3_thinking_level_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

API Parameters:

thinking_level: Set to "high" (for Deep Think) or "low" (for faster, cheaper responses).

include_thoughts: Set to True to get the model's intermediate thinking steps.

In [4]:
# 1. IMPORTS & SETUP
import google.genai as genai
from google.genai import types
import os

# --- Secure API Key Retrieval ---
# NOTE: If you are still seeing the ValidationError, you MUST run:
# !pip install --upgrade google-genai
# and restart your Python runtime (e.g., in Colab, go to Runtime -> Restart runtime).

GEMINI_API_KEY = None
try:
    # Attempt to load from Colab secrets ('GEMINI')
    from google.colab import userdata
    GEMINI_API_KEY = userdata.get('GEMINI')
except (ImportError, KeyError):
    # Fallback to standard environment variable
    GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')

REQUESTED_MODEL_ID = 'gemini-3-pro-preview'
client = None

if GEMINI_API_KEY:
    try:
        client = genai.Client(api_key=GEMINI_API_KEY)
        print(f"✅ Gemini client configured for **{REQUESTED_MODEL_ID}**.")
    except Exception as e:
        print(f"❌ Client initialization failed: {e}")
else:
    print("❌ API Key not found. Please ensure your key is set up.")

# 2. DEEP THINK CONFIGURATION & API CALL
if client:
    # CORRECTED CONFIGURATION: Use ThinkingConfig object for Gemini 3
    deep_think_config = types.GenerateContentConfig(
        thinking_config=types.ThinkingConfig(
            # 'high' is the Deep Think level for maximum deliberation
            thinking_level="high",

            # This is essential to request the thought summary
            include_thoughts=True
        )
    )

    # Define a prompt requiring multi-step, complex reasoning
    complex_prompt = (
        "Solve the following logic puzzle: Three people, Alice, Bob, and Carol, have 3 hats (2 red, 1 blue). "
        "They are lined up, and each can only see the hats of the people in front of them. "
        "The last person (Carol) says, 'I cannot tell what color hat I am wearing.' "
        "The middle person (Bob) then says, 'I cannot tell what color hat I am wearing.' "
        "What color hat is the first person (Alice) wearing, and why?"
    )

    print("\n--- Sending request with Deep Think mode ---")

    # Execute the API call
    response = client.models.generate_content(
        model=REQUESTED_MODEL_ID,
        contents=complex_prompt,
        config=deep_think_config
    )

    # 3. ROBUST DISPLAY LOGIC (FIXED)

    final_answer = ""
    thought_summary = ""

    # Iterate through the parts of the content, as thought summaries are embedded here.
    if response.candidates and response.candidates[0].content:
        for part in response.candidates[0].content.parts:
            # The 'thought' boolean flag indicates a thought summary part
            if hasattr(part, 'thought') and part.thought and part.text:
                thought_summary += part.text + "\n"
            elif part.text:
                final_answer += part.text

    # Display Results
    print("\n=======================================================")
    print("🧠 FINAL ANSWER (Deep Think Result):")
    print(final_answer.strip())

    if thought_summary:
        print("\n--- Model's Internal Deep Thought Process ---")
        print(thought_summary.strip())
    else:
        print("\n--- Internal Deep Thought Process not available in response content. ---")

✅ Gemini client configured for **gemini-3-pro-preview**.

--- Sending request with Deep Think mode ---

🧠 FINAL ANSWER (Deep Think Result):
Alice is wearing a **Red** hat.

Here is the step-by-step logical deduction:

1.  **Analyze the Inventory:** There are only 3 hats total: **2 Red** and **1 Blue**.

2.  **Analyze Bob's Statement:**
    *   Bob can see Alice's hat.
    *   Bob says, "I cannot tell what color hat I am wearing."
    *   **The Logic:** If Alice were wearing the **Blue** hat, Bob would see it. Since there is only *one* Blue hat in existence, Bob would immediately know that his own hat *must* be Red.
    *   Because Bob does *not* know his hat color, he must not be seeing a Blue hat on Alice.

3.  **Conclusion:**
    *   Since Bob does not see a Blue hat on Alice, she must be wearing a **Red** hat.

*(Note: While Carol's statement is part of the puzzle's narrative, Bob's statement alone provides sufficient proof. In this specific variation with only 1 Blue hat, if Alice 

In [7]:
# 1. IMPORTS & API SETUP
import google.genai as genai
from google.genai import types
import os

# --- Secure API Client Initialization using Userdata/Environment Variables ---
GEMINI_API_KEY = None
try:
    # Attempt to load from Colab secrets ('GEMINI')
    # Note: This will only work if run in a Colab environment with a secret named 'GEMINI'
    from google.colab import userdata
    GEMINI_API_KEY = userdata.get('GEMINI')
except (ImportError, KeyError):
    # Fallback to standard environment variable
    GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')

REQUESTED_MODEL_ID = 'gemini-3-pro-preview'
client = None

if GEMINI_API_KEY:
    try:
        client = genai.Client(api_key=GEMINI_API_KEY)
        print(f"✅ Gemini client configured for **{REQUESTED_MODEL_ID}**.")
    except Exception as e:
        print(f"❌ Client initialization failed: {e}")
else:
    print("❌ API Key not found. Please ensure 'GEMINI' secret is set in Colab or GEMINI_API_KEY is in your environment.")


# --- 2. AGENT CONFIGURATIONS ---

# 🎯 Agent 1: High-Throughput Agent (Low Thinking)
def get_low_think_config():
    """Returns config for fast, low-cost operations (e.g., classification, data extraction)."""
    return types.GenerateContentConfig(
        thinking_config=types.ThinkingConfig(
            # Minimizes internal deliberation, resulting in lower latency and cost.
            thinking_level="low"
        )
    )

# 🎯 Agent 2: Deep Reasoning Agent (High Thinking / Deep Think Mode)
def get_high_think_config():
    """Returns config for rigorous, multi-step reasoning (e.g., complex optimization, compliance)."""
    return types.GenerateContentConfig(
        thinking_config=types.ThinkingConfig(
            # Maximizes internal deliberation for complex, high-stakes decisions.
            thinking_level="high",
            # Request internal thoughts for auditability of the complex reasoning chain.
            include_thoughts=True
        )
    )

# --- 3. MULTI-AGENT ORCHESTRATOR SIMULATION ---

def aocc_orchestrator(system_prompt: str):
    """Simulates the AOCC orchestrator delegating tasks to specialized agents."""

    if not client:
        print("\nOrchestrator cannot run without a configured Gemini client.")
        return

    print("\n=======================================================")
    print(f"Orchestrator receives operational query: {system_prompt}\n")

    # Task 1: Simple, High-Throughput Check (Delegated to Low-Think Agent)
    # Goal: Quickly classify an event or extract key data points.
    low_think_prompt = (
        "Classify the severity of this event (Low, Medium, High) and extract the flight number: "
        "A ground crew reported a 30-minute delay due to a baggage belt failure for flight UA456. "
        "Output ONLY a single JSON object with keys 'flight_number' and 'severity'."
    )

    print("--- 🟢 DELEGATING TO LOW-THINK AGENT (Classification/Fast Data) ---")
    response_low = client.models.generate_content(
        model=REQUESTED_MODEL_ID,
        contents=low_think_prompt,
        config=get_low_think_config()
    )
    print(f"Agent 1 Result (LOW THINK): {response_low.text.strip()}")
    print("-" * 50)

    # Task 2: Complex, High-Stakes Optimization (Delegated to High-Think Agent)
    # Goal: Analyze multiple conflicting constraints to produce a strategic solution.
    high_think_prompt = (
        "We need to find a new gate for diverted flight BA789 (Large aircraft). "
        "It requires a heavy jet gate, but all are currently occupied by flights delayed by more than 4 hours. "
        "Propose the optimal 3-step solution to minimize overall passenger impact (connecting passenger misses). "
        "Detail the reasoning for de-prioritizing one of the currently occupied gates."
    )

    print("--- 🔴 DELEGATING TO HIGH-THINK AGENT (Optimization/Complex Reasoning) ---")
    response_high = client.models.generate_content(
        model=REQUESTED_MODEL_ID,
        contents=high_think_prompt,
        config=get_high_think_config()
    )

    # Extract Thought Summary for High-Think Agent for auditability
    thought_summary = next((part.text for part in response_high.candidates[0].content.parts if hasattr(part, 'thought') and part.thought), None)

    print(f"Agent 2 Result (HIGH THINK):\n{response_high.text.strip()}")

    if thought_summary:
        print("\n*** DEEP THINK INTERNAL REASONING (Audit Trail from Agent 2) ***")
        # Assuming the thought structure is a JSON or similar that needs simple printing.
        print(thought_summary.strip())

    print("=======================================================")

# --- EXECUTE ORCHESTRATION ---
aocc_orchestrator("Process a new gate request and a small baggage delay event.")

✅ Gemini client configured for **gemini-3-pro-preview**.

Orchestrator receives operational query: Process a new gate request and a small baggage delay event.

--- 🟢 DELEGATING TO LOW-THINK AGENT (Classification/Fast Data) ---
Agent 1 Result (LOW THINK): ```json
{
  "flight_number": "UA456",
  "severity": "Low"
}
```
--------------------------------------------------
--- 🔴 DELEGATING TO HIGH-THINK AGENT (Optimization/Complex Reasoning) ---
Agent 2 Result (HIGH THINK):
Here is the optimal 3-step solution to secure a gate for flight BA789, designed to minimize overall passenger impact and connection misses.

### The 3-Step Solution

**Step 1: Identification and Towing (The "Swap")**
Identify the delayed heavy jet currently occupying a gate that has the **longest Estimated Time of Departure (ETD)** or is awaiting a specific non-immediate resource (e.g., a replacement part or fresh crew arriving in >1 hour). Immediately tow this aircraft to a remote hardstand (parking spot).
*   *Note:* If